In [1]:
import pandas as pd

# Load CSV file

url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)

#Look at the first few rows
df.head()

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
0,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,IE0,Ireland,Number,5149139
1,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1492-13a3-e055-000000000001,Carlow County Council,Number,61968
2,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1433-13a3-e055-000000000001,Dublin City Council,Number,592713
3,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-149f-13a3-e055-000000000001,Dún Laoghaire Rathdown County Council,Number,233860
4,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-14a0-13a3-e055-000000000001,Fingal County Council,Number,330506


In [ ]:
# Convert the 'Single year of Age' and 'Value' columns to numbers
# errors="coerce" will urn any bad values into NaN
df["Single Year of Age"] = pd.to_numeric(df["Single Year of Age"], errors="coerce")
df["VALUE"] = pd.to_numeric(df["VALUE"], errors="coerce")

# Drop any rows where age or value is missing
df = df.dropna(subset=["Single Year of Age", "VALUE"])

In [ ]:
# Check the column names to see what can be used
df.columns

Index(['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear',
       'C02199V02655', 'Sex', 'C02076V03371', 'Single Year of Age',
       'C03789V04537', 'Administrative Counties', 'UNIT', 'VALUE'],
      dtype='object')

In [3]:
# Look at the unique values in the 'SEX' column
df["Sex"].unique()

array(['Both sexes', 'Male', 'Female'], dtype=object)

In [4]:
# Select only males
males = df[df["Sex"] == "Male"]

# Total male population
total_males = males["VALUE"].sum()

# Sum of age and population for males
male_age_times_pop = (males["Single Year of Age"] * males["VALUE"]).sum()

# Weighted mean age for males
male_weighted_mean_age = male_age_times_pop / total_males

male_weighted_mean_age

TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# Select only females
females = df[df["Sex"] == "Female"]

# Total female population
total_females = females["VALUE"].sum()

# Sum of age and population for females
female_age_times_pop = (females["Single Year of Age"] * females["VALUE"]).sum()

# Weighted mean age for females
female_weighted_mean_age = female_age_times_pop / total_females

female_weighted_mean_age

/tmp/ipykernel_10604/3785488419.py:11: RuntimeWarning: invalid value encountered in scalar divide
  female_weighted_mean_age = female_age_times_pop / total_females


np.float64(nan)

In [23]:
male_by_age = males.groupby("Single Year of Age")["VALUE"].sum()
females_by_age = females.groupby("Single Year of Age")["VALUE"].sum()

difference_by_age = male_by_age - females_by_age
difference_by_age

Series([], Name: VALUE, dtype: int64)